In [1]:
!pip install finterstellar

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------  3.9/4.0 MB 36.6 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 12.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------------------------------------- -- 7.6/8.1 MB 36.9 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 31.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 23.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------  6.8/7.0 MB 36.9 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 30.1 MB/s eta 0:00:00

   ---- ----------------------------------- 1/9 [pillow]
   ---- ----------------------------------- 1/9 [pillow]
   ---- ----------------------------------- 1/9 [pillow]
   -------- -

In [2]:
# Import library
import finterstellar as fs
#System trading Library 설치: Finterstellar

import io, sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
symbol_ = 'TSLA'
start_date_ = '2020-06-30'
end_date_ = '2025-06-30'
fee_ = 0.001

In [4]:
path_list = ["rsi","macd","stochastic"]

for path in path_list :
    try:
        os.makedirs(path, exist_ok=True)
        print(f"{path} 디렉토리가 생성되었거나 이미 존재합니다.")
    except OSError as e:
        print(f"디렉토리 생성 중 오류 발생: {e}")


rsi 디렉토리가 생성되었거나 이미 존재합니다.
macd 디렉토리가 생성되었거나 이미 존재합니다.
stochastic 디렉토리가 생성되었거나 이미 존재합니다.


In [5]:
# Performance 문자열을 Parsing 하여 반환하는 함수
def parse_performance_output(output_text):
    result = {}
    for line in output_text.strip().split("\n"):
        # 콜론으로 분리
        if ":" not in line:
            continue
        key, value = line.split(":", 1)
        key = key.strip()
        value = value.strip()

        # 퍼센트 처리
        if value.endswith("%"):
            num = float(value.strip("%")) / 100
            result[key] = num
        # 연도 처리
        elif value.endswith("yrs"):
            num = float(value.strip("yrs"))
            result[key] = num
        # 숫자 처리
        else:
            try:
                result[key] = float(value)
            except ValueError:
                result[key] = value  # 혹시 문자열이면 그대로 저장

    return result

In [6]:
# 1. RSI Performance and evaluation

In [7]:
# RSI 를 구해서 dict 를 반환하는 함수
def get_rsi_performance_dict(symbol_, df, params) :
    w_, buy_, sell_, fee_ = params

    # dataFrame 초기화
    df_tmp = df.copy()

    result_dict = {}

    # 파라미터 추가
    result_dict["symbol"] = symbol_
    result_dict["w_param"] = w_
    result_dict["buy_param"] = buy_
    result_dict["sell_param"] = sell_
    result_dict["fee_"] = fee_

    perf = {}

    # RSI 계산, indicator signal 도출, position 계산, 평가 및 performance 측정
    fs.rsi(df_tmp, w=w_)
    fs.indicator_to_signal(df_tmp, factor='rsi', buy=buy_, sell=sell_)
    fs.position(df_tmp)
    fs.evaluate(df_tmp, cost= fee_)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # Performance 함수 실행 (print만 함)
        fs.performance(df_tmp, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    perf = parse_performance_output(output_text)

    result_dict.update(perf)

    print(f"Performance Processing : {result_dict}")

    return result_dict

In [8]:
# MACD 를 구해서 dict 를 반환하는 함수
def get_macd_performance_dict(symbol_, df, params) :
    buy_, sell_, fee_ = params

    # dataFrame 초기화
    df_tmp = df.copy()

    result_dict = {}

    # 파라미터 추가
    result_dict["symbol"] = symbol
    result_dict["buy_param"] = buy_
    result_dict["sell_param"] = sell_
    result_dict["fee_"] = fee_

    perf = {}

    # MACD 계산, indicator signal 도출, position 계산, 평가 및 performance 측정
    fs.macd(df_tmp)
    fs.indicator_to_signal(df_tmp, factor='macd', buy=buy_, sell=sell_)
    fs.position(df_tmp)
    fs.evaluate(df_tmp, cost= fee_)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # Performance 함수 실행 (print만 함)
        fs.performance(df_tmp, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    perf = parse_performance_output(output_text)

    result_dict.update(perf)

    print(f"Performance Processing : {result_dict}")

    return result_dict

In [9]:
# Stochastic 를 구해서 dict 를 반환하는 함수
def get_stochastic_performance_dict(symbol_, df, params) :
    factor_, buy_, sell_, fee_ = params

    # dataFrame 초기화
    df_tmp = df.copy()

    result_dict = {}

    # 파라미터 추가
    result_dict["symbol"] = symbol
    result_dict["factor"] = factor_
    result_dict["buy_param"] = buy_
    result_dict["sell_param"] = sell_
    result_dict["fee_"] = fee_

    perf = {}

    # Stochastic 계산, indicator signal 도출, position 계산, 평가 및 performance 측정
    fs.stochastic(df_tmp, symbol, n=14, m=3, t=3)
    # indicator 값 추출을 위한 전처리 작업
    df_tmp['indicator'] = df_tmp['slow_k'] - df_tmp['slow_d']

    fs.indicator_to_signal(df_tmp, factor= factor_, buy=buy_, sell=sell_)
    fs.position(df_tmp)
    fs.evaluate(df_tmp, cost= fee_)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # Performance 함수 실행 (print만 함)
        fs.performance(df_tmp, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    perf = parse_performance_output(output_text)

    result_dict.update(perf)

    print(f"Performance Processing : {result_dict}")


    return result_dict

In [12]:
# 모든 팩터 집합을 이용해 최적의 RSI Parameter 계산
# Parallel 을 이용해서 성능 개선
from joblib import Parallel, delayed
from tqdm import tqdm

def get_rsi_performance_dataframe(symbol, start_date_, end_date_, fee_) :

    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    print(f"[ {symbol} ] dataFrame.head ")
    print(df.head(3))

    # Parameter 집합 도출(list)
    w_range = range(7,23+1)
    buy_range = range(15,45+1)
    sell_range = range(55,85+1)
    param_list = [(w,buy,sell,fee_) for w in w_range for buy in buy_range for sell in sell_range]

    results = []

    # 최적의 w, buy_position, sell_position 을 찾기 위한 recursive
    # For문 이용
    # for w in range(7,23+1)
    #   for buy in range(15, 45+1)
    #       for sell in range(55,85+1)
    #           results.append(get_rsi_performance_dict(df,w,buy,sell,fee_))

    # Parallel 이용
    results = Parallel(n_jobs=-1) (
            delayed(get_rsi_performance_dict)(symbol, df, params) for params in tqdm(param_list)

    )

    df_result = pd.DataFrame(results)
    return df_result

In [13]:
# 모든 팩터 집합을 이용해 최적의 MACD Parameter 계산
# Parallel 을 이용해서 성능 개선
from joblib import Parallel, delayed
from tqdm import tqdm

def get_macd_performance_dataframe(symbol, start_date_, end_date_, fee_) :

    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    print(f"[ {symbol} ] dataFrame.head ")
    print(df.head(3))

    # Parameter 집합 도출(list)
    buy_range = range(-10,10+1)
    sell_range = range(-10,10+1)
    param_list = [(buy,sell,fee_) for buy in buy_range for sell in sell_range]

    results = []

    # 최적의 buy_position, sell_position 을 찾기 위한 recursive
    # For문 이용
    # for buy in range(-10,10+1)
    #    for sell in range(-10,10+1)
    #        results.append(get_macd_performance_dict(df,buy,sell,fee_))

    # Parallel 이용
    results = Parallel(n_jobs=-1) (
            delayed(get_macd_performance_dict)(symbol, df, params) for params in tqdm(param_list)

    )

    df_result = pd.DataFrame(results)
    return df_result

In [14]:
# 모든 팩터 집합을 이용해 최적의 Stochastic Parameter 계산
# Parallel 을 이용해서 성능 개선
from joblib import Parallel, delayed
from tqdm import tqdm

def get_stochastic_performance_dataframe(symbol, start_date_, end_date_, fee_) :

    df = fs.get_ohlc(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_ohlc_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    print(f"[ {symbol} ] dataFrame.head ")
    print(df.head(3))

    # Parameter 집합 도출(list)
    factor_list = ['slow_k','indicator']
    buy_range = range(10,40+1)
    sell_range = range(60,90+1)
    param_list = [(factor_,buy,sell,fee_) for factor_ in factor_list for buy in buy_range for sell in sell_range]

    results = []

    # 최적의 buy_position, sell_position 을 찾기 위한 recursive
    # For문 이용
    # for factor in factor_list
    #   for buy in range(10, 40+1)
    #       for sell in range(60,90+1)
    #           results.append(get_stochastics_performance_dict(factor,buy,sell,fee_))

    # Parallel 이용
    results = Parallel(n_jobs=-1) (
            delayed(get_stochastics_performance_dict)(symbol, df, params) for params in tqdm(param_list)
    )

    df_result = pd.DataFrame(results)
    return df_result

In [15]:
symbol_list = ['^GSPC', '^KS11','MSFT', 'GOOG', 'AMZN', 'META', 'NVDA', 'TSLA', 'AAPL']

In [16]:
# Parameter 조합을 csv 로 저장
for symbol in symbol_list :
    df_rsi = get_rsi_performance_dataframe(symbol, start_date_, end_date_, fee_)
    df_macd = get_macd_performance_dataframe(symbol, start_date_, end_date_, fee_)
    df_sto = get_stochastic_performance_dataframe(symbol, start_date_, end_date_, fee_)

    # write csv
    df_rsi.to_csv(f'./rsi/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)
    df_macd.to_csv(f'./macd/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)
    df_sto.to_csv(f'./stochastic/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)

[ ^GSPC ] dataFrame.head 
              ^GSPC
2020-06-30 3,100.29
2020-07-01 3,115.86
2020-07-02 3,130.01


  0%|          | 48/16337 [00:14<1:17:38,  3.50it/s]

KeyboardInterrupt: 

In [ ]:
# Read_csv
# Max CAGR, Max Sharpe Ratio 값 구하기
# 해당 값에 해당하는 performance 값 계산 후 image 출력

In [ ]:
# Max Parameter 를 이용하여 chart Graph 도출
def get_rsi_max_performance(symbol, start_date_, end_date_, fee_, text) :
    df_rsi = pd.read_csv(f'./rsi/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)

    max_series = df_rsi[df_rsi[text] == df_rsi[text].max()].iloc[0]
    max_w = max_series['w_param']
    max_buy = max_series['buy_param']
    max_sell = max_series['sell_param']

    print(f"### {symbol}'s Parameter of max({text})")
    print(f"{max_series}")

    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    # rsi
    fs.rsi(df, w=max_w)
    fs.draw_chart(df, left='rsi', right= symbol)
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_price_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor='rsi', buy=max_buy, sell=max_sell)
    # Calc position
    fs.position(df)
    fs.draw_chart(df, left='rsi', right='position_chart')
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    # Evalution
    fs.evaluate(df, cost= fee_)
    fs.draw_chart(df, left='acc_rtn_dp', right=symbol)
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_evaluation_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    # get Performance
    fs.performance(df, rf_rate=0.02)

    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()


In [ ]:
# Max Parameter 를 이용하여 chart Graph 도출
def get_macd_max_performance(symbol, start_date_, end_date_, fee_, text) :
    df_macd = pd.read_csv(f'./macd/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)

    max_series = df_macd[df_macd[text] == df_macd[text].max()].iloc[0]
    max_buy = max_series['buy_param']
    max_sell = max_series['sell_param']

    print(f"### {symbol}'s Parameter of max({text})")
    print(f"{max_series}")

    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    # macd
    fs.macd(df)
    fs.draw_chart(df, right=['macd','macd_signal','macd_oscillator'])
    # 파일로 저장
    plt.savefig(f"./macd/{symbol}_{text}_price_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor='macd_oscillator', buy=max_buy, sell=max_sell)

    # Calc position
    fs.position(df)
    fs.draw_chart(df, right='position_chart', left='macd_oscillator')
    # 파일로 저장
    plt.savefig(f"./macd/{symbol}_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()


    # Evalution
    fs.evaluate(df, cost=fee_)
    fs.performance(df, rf_rate=0.02)
    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"./macd/{symbol}_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

In [ ]:
# Max Parameter 를 이용하여 chart Graph 도출
def get_stochastic_max_performance(symbol, start_date_, end_date_, fee_, text) :
    df_sto = pd.read_csv(f'./stochastic/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)

    max_series = df_sto[df_sto[text] == df_sto[text].max()].iloc[0]
    max_factor = max_series['factor']
    max_buy = max_series['buy_param']
    max_sell = max_series['sell_param']

    print(f"### {symbol}'s Parameter of max({text})")
    print(f"{max_series}")

    df = fs.get_ohlc(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_ohlc_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    # stochastic
    fs.stochastic(df, symbol, n=14, m=3, t=3)
    df['indicator'] = df['slow_k'] - df['slow_d']
    fs.draw_chart(df, left=max_factor, right=symbol)
    # 파일로 저장
    plt.savefig(f"./stochastic/{symbol}_{text}_price_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor=max_factor, buy=max_buy, sell=max_sell)

    # Calc position
    fs.position(df)
    fs.draw_chart(df, right='position_chart', left='slow_k')
    # # 파일로 저장
    plt.savefig(f"./stochastic/{symbol}_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # # 리소스 해제
    # plt.close()


    # Evalution
    fs.evaluate(df, cost=fee_)
    fs.performance(df, rf_rate=0.02)
    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"./stochastic/{symbol}_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

In [ ]:
cols = ['CAGR','Sharpe ratio']

for symbol in symbol_list :
    for col in cols :
        get_rsi_max_performance(symbol, start_date_, end_date_, fee_, col)
        get_macd_max_performance(symbol, start_date_, end_date_, fee_, col)
        get_stochastic_max_performance(symbol, start_date_, end_date_, fee_, col)